In [ ]:
import numpy as np

np.random.seed(42)

n_players = 10    # 타자 수
at_bats = 50      # 타석 수

players = {}
for i in range(n_players):
    players[i] = {
        'OBP': np.random.uniform(0.25, 0.45, at_bats),
        'SLG': np.random.uniform(0.30, 0.60, at_bats),
        'BABIP': np.random.uniform(0.25, 0.4, at_bats),
        'BB_K': np.random.uniform(0.2, 1.5, at_bats),
        'ISO': np.random.uniform(0.05, 0.25, at_bats),
    }

# 실제 안타 여부 생성 (베르누이)
true_coef = np.array([2.0, 3.0, 2.5, 1.0, 1.5])
bias = -3.0
for i in range(n_players):
    X = np.column_stack([players[i]['OBP'], players[i]['SLG'],
                         players[i]['BABIP'], players[i]['BB_K'],
                         players[i]['ISO']])
    linear = np.dot(X, true_coef) + bias
    prob = 1 / (1 + np.exp(-linear))
    players[i]['hit'] = np.random.binomial(1, prob)

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def logistic_regression(X, y, lr=0.1, epochs=1000):
    n_samples, n_features = X.shape
    w = np.zeros(n_features)
    b = 0
    for _ in range(epochs):
        linear = np.dot(X, w) + b
        y_pred = sigmoid(linear)
        dw = np.dot(X.T, (y_pred - y)) / n_samples
        db = np.sum(y_pred - y) / n_samples
        w -= lr * dw
        b -= lr * db
    return w, b


all_X = []
all_y = []
for i in range(n_players):
    X = np.column_stack([players[i]['OBP'], players[i]['SLG'],
                         players[i]['BABIP'], players[i]['BB_K'],
                         players[i]['ISO']])
    y = players[i]['hit']
    all_X.append(X)
    all_y.append(y)
all_X = np.vstack(all_X)
all_y = np.hstack(all_y)

weights, bias = logistic_regression(all_X, all_y, lr=0.5, epochs=1000)
print("학습된 가중치:", weights)
print("학습된 편향:", bias)

print("\n=== 타자별 신타율 ===")
for i in range(n_players):
    X = np.column_stack([players[i]['OBP'], players[i]['SLG'],
                         players[i]['BABIP'], players[i]['BB_K'],
                         players[i]['ISO']])
    pred_prob = sigmoid(np.dot(X, weights) + bias)
    new_avg = np.mean(pred_prob)  # 신타율
    print(f"타자 {i+1} 신타율: {new_avg:.3f}")
